# Problem 22

In [1]:
M = MatrixSpace(QQ, 4, 4, sparse=False)
V = VectorSpace(QQ, 4)
show(latex(M))
A = M([-19, 8, 96, 48, 64, 49, 16, 8, 32, 18, 21, 4, -44, -28, -24, 1])
show(LatexExpr("A :="), A)

\mathrm{Mat}_{4\times 4}(\Bold{Q})

A := [-19   8  96  48]
[ 64  49  16   8]
[ 32  18  21   4]
[-44 -28 -24   1]

Show that $A$ is diagonalizable then find its eigendecomposition $A = KDK^{-1}$ 

## Step 1: Show diagonalizability

$A \text{ is diagonalizable} \iff \forall\lambda \in \sigma(A), a.m.(\lambda) = g.m.(\lambda)$

In [2]:
showp = lambda x, y: show(LatexExpr(x), y)
l = var('l', latex_name=r'\lambda')
# compute A - lambda * I
AmlI = A - l*M.identity_matrix()

# Create characteristic polynomial
p(l) = AmlI.determinant()

showp(r"\text{Characteristic polynomial of } A:" ,p(l).expand())

\text{Characteristic polynomial of } A: l^4 - 52*l^3 - 1690*l^2 + 61516*l - 428415

In [3]:
show(p.factor())

l |--> (l + 39)*(l - 13)^2*(l - 65)

The eigenvalues are: $-39, 13, 13, 65$, and their algebraic multiplicites are $1,2,2,1$.

We now find the eigenvectors by solving the equation below:

In [4]:
w, x, y, z = var('w,x,y,z')
show(AmlI, matrix([[w], [x], [y], [z]]), LatexExpr("="), matrix([[0], [0], [0], [0]]))

[-l - 19       8      96      48]
[     64 -l + 49      16       8]
[     32      18 -l + 21       4]
[    -44     -28     -24  -l + 1] [w]
[x]
[y]
[z] = [0]
[0]
[0]
[0]

In [7]:
def compute_eigenvectors(mat, eigenval):
    L = mat(l=eigenval)

    Lwxyz = L * matrix([[w], [x], [y], [z]])
    sol = solve([p[0] for p in Lwxyz], w, x, y ,z)
    dummy_vars = set()
    for s in sol[0]:
        dummy_vars = dummy_vars.union(set(s.variables()))
    dummy_vars -= set([w,x,y,z])
    # dummy_vars = set([*sol[0][0].variables(), *sol[0][1].variables(), *sol[0][2].variables(), *sol[0][3].variables()]) - set([x, y, z, w])
    show(LatexExpr(r"\lambda = " + str(eigenval)))
    if len(dummy_vars) > 1:
        subs_map = {}
        for i, dummy in enumerate(dummy_vars):
            subs_map.update({dummy: var(f'v{i+1}')})
        print(f"Geometric multiplicity is: {len(dummy_vars)}")
        sol = vector([s.subs(subs_map).rhs() for s in sol[0]])
        show(sol)
        return sol, len(subs_map.keys())
    else:
        v1 = var('v1')
        dummy = dummy_vars.pop()
        sol = vector([s.subs({dummy: v1}).rhs() for s in sol[0]])
        show(sol)
        return sol, 1
    
el1, multiplicity1 = compute_eigenvectors(AmlI, 65)
el2, multiplicity2 = compute_eigenvectors(AmlI, -39)
el3, multiplicity3 = compute_eigenvectors(AmlI, 13)

print("Eigenvectors computed")

assert multiplicity1 == 1
assert multiplicity2 == 1
assert multiplicity3 == 2
assert A.is_diagonalizable() # sanity check for my code :P

print("Asserts passed, matrix is diagonalizable")

\lambda = 65

(-2/7*v1, -9/7*v1, -9/14*v1, v1)

\lambda = -39

(5/2*v1, -7/4*v1, -7/8*v1, v1)

\lambda = 13

Geometric multiplicity is: 2


(2*v1 + v2, -4*v1 - 2*v2, v1, v2)

Eigenvectors computed
Asserts passed, matrix is diagonalizable


In [ ]:
# Check with Sage solution
assert A * el1 == 65 * el1
assert A * el2 == -39 * el2
# lambda 13, vector 1
assert A * el3(v1=0) == A * el3(v1=0)
# lambda 13, vector 2
assert A * el3(v2=0) == A * el3(v2=0)
print("Asserts passed, eigenvectors are correct")

Now that we have all the eigenvalues and eigenvectors, we can build $K, D$ and $K^{-1}$. $K$'s columns are
eigenvectors, and $D$ is a diagonal matrix whose diagonal entries are the eigenvalues of the matrix

In [ ]:
D = M.diagonal_matrix([65, -39, 13, 13])
# I'm rescaling below to fit the eigenvectors in TAM_ex.pdf
K = M([el1(v1=7 * 4/2), el2(v1=2 * 20 / 5), el3(v1=1,v2=0), el3(v1=0,v2=1)]).T
showp("K =", K)
showp("D =", D)

We only know K and D, so we need to compute $K^{-1}$ and $D^{-1}$. We'll invert K using the adjunct matrix method:

$$
    A^{-1} = \frac{1}{det(A)}C^T
$$

Where $C$ is the cofactor matrix of A. $C^T = \text{adj}(A)$

First, let's compute the determinant of K. We simplify the matrix first so that we only need to compute only one the minors of the first row instead of the usual 4.

In [ ]:
showm = lambda x: show(LatexExpr(x))

Ksimplified = copy(K)
showm(r"C_4 \leftarrow C_4 - 2C_3")
Ksimplified[:, 3] -= 2 * Ksimplified[:, 2]
show(Ksimplified)
showm(r"C_2 \leftarrow C_2 - 20C_3")
Ksimplified[:, 1] -= 20 * Ksimplified[:, 2]
show(Ksimplified)
showm(r"C_1 \leftarrow C_1 + 4C_3")
Ksimplified[:, 0] += 4 * Ksimplified[:, 2]
show(Ksimplified)

In [ ]:
def minor(mat, i, j):
    """Compute the minor of a given matrix at row i and column j"""
    return mat.delete_rows([i]).delete_columns([j]).determinant()

detK = pow(-1, 2) * minor(Ksimplified, 0, 2) # Because other 1st row entries are 0, we only use the minor in the 3rd column
assert detK == Ksimplified.determinant()

In [ ]:
def cofactor(mat):
    """Compute the cofactor matrix"""
    result = M()
    for i in range(0, mat.nrows()):
        for j in range(0, mat.ncols()):
            result[i,j] = pow(-1,(i+j)) * minor(mat, i, j)
    return result

def adj(mat):
    """Compute the adjunct matrix of mat"""
    return cofactor(mat).T

adjK = adj(K)
assert adjK == K.adjugate()

In [ ]:
invK = adjK / detK
assert invK == K.inverse()

The solution to the exercise is:
$A = KDK^{-1}$

In [ ]:
show(A, LatexExpr("="), K, D, invK)